#### Companion reading if you want to understand how it works:  

[Creating a (mostly) Autonomous HR Assistant with ChatGPT and LangChain’s Agents and Tools](https://medium.com/@stephen.bonifacio/creating-a-mostly-autonomous-hr-assistant-with-chatgpt-and-langchains-agents-and-tools-1cdda0aa70ef)



In [1]:
# load core modules
from langchain.chains import RetrievalQA
# load agents and tools modules
import pandas as pd
from io import StringIO
#from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain import LLMMathChain
from langchain_chroma import Chroma
from langchain_groq import ChatGroq



In [2]:
from sentence_transformers import SentenceTransformer

# Load the sentence-transformers model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define the custom embed_query method with conversion to list
def embed_query(self, texts):
    """
    Custom embedding function to handle query embedding.
    Ensures that embeddings are returned as a list of lists.
    """
    embeddings = self.encode(texts)  # Get embeddings as ndarray
    return embeddings.tolist()  # Convert ndarray to list of lists

# Add the embed_query method to the SentenceTransformer object
import types
model.embed_query = types.MethodType(embed_query, model)

vector_store = Chroma(
    collection_name="policies",
    persist_directory="./vectorstore",
    embedding_function=model,
)

c:\Users\SivaSaiNS\Proj\HR - Chatbot\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\SivaSaiNS\Proj\HR - Chatbot\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:


# initialize LLM object
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="mixtral-8x7b-32768",
    max_retries=2,
)

# initialize vectorstore retriever object
timekeeping_policy = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
)

df = pd.read_csv("employee_data.csv") # load employee_data.csv as dataframe
class CustomPythonTool:
    def __init__(self, df):
        self.df = df

    def run(self, query):
        try:
            # Execute query using the df
            result = eval(query, {"df": self.df})
            return str(result)
        except Exception as e:
            return f"Error: {str(e)}"

# Initialize your custom tool
python_tool = CustomPythonTool(df)
#python = PythonAstREPLTool(locals={"df": df}) # set access of python_repl tool to the dataframe

# create calculator tool
calculator = LLMMathChain.from_llm(llm=llm, verbose=True)

# create variables for f strings embedded in the prompts
user = 'Siva Sai' # set user
df_columns = df.columns.to_list() # print column names of df

# prep the (tk policy) vectordb retriever, the python_repl(with df access) and langchain calculator as tools for the agent
tools = [
    Tool(
        name = "Timekeeping Policies",
        func=timekeeping_policy.run,
        description="""
        Useful for when you need to answer questions about employee timekeeping policies.

        <user>: What is the policy on unused vacation leave?
        <assistant>: I need to check the timekeeping policies to answer this question.
        <assistant>: Action: Timekeeping Policies
        <assistant>: Action Input: Vacation Leave Policy - Unused Leave
        ...
        """
    ),
    Tool(
        name = "Employee Data",
        func=python_tool.run,
        description = f"""
        Useful for when you need to answer questions about employee data stored in pandas dataframe 'df'. 
        Run python pandas operations on 'df' to help you get the right answer.
        'df' has the following columns: {df_columns}
        
        Question: How many Sick Leave do I have left?
        Thought: I need to check the employee data to get this information.
        Action: Employee Data
        Action Input: df[df['name'] == 'Siva Sai']['sick_leave']
        Observation: 45
        Thought: I now know the final answer.
        Final Answer: You have 45 sick leaves left.
        """
    ),
    Tool(
        name = "Calculator",
        func=calculator.run,
        description = f"""
        Useful when you need to do complex math operations or arithmetic.
        Question: Calculate 30 times my basic pay divided by 30.
        Thought: I need to first fetch the basic pay and then use the calculator to perform the math operation.
        Action: Employee Data
        Action Input: df[df['name'] == 'Siva Sai']['basic_pay_in_rs'].iloc[0]
        Observation: 50000  # Assuming the basic pay is Rs. 50,000
        Thought: I now have the basic pay and can proceed with the calculation.
        Action: Calculator
        Action Input: (30 * 50000 / 30)
        Observation: 50000
        Thought: I now know the final answer.
        Final Answer: The result of the calculation is 50,000.
        """
    )
]

# change the value of the prefix argument in the initialize_agent function. This will overwrite the default prompt template of the zero shot agent type
agent_kwargs = {'prefix': f'You are friendly HR assistant. You are tasked to assist the current user: {user} on questions related to HR. If you do not find any relavent data using below tools then say i do not know. You have access to the following tools:'}


# initialize the LLM agent
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True, 
                         agent_kwargs=agent_kwargs
                         )
# define q and a function for frontend
def get_response(user_input):
    response = agent.invoke(user_input)
    return response

In [4]:
print(agent.agent.llm_chain.prompt.template)

You are friendly HR assistant. You are tasked to assist the current user: Siva Sai on questions related to HR. If you do not find any relavent data using below tools then say i do not know. For general questions no need to use tools. You have access to the following tools:

Timekeeping Policies(*args: Any, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - 
        Useful for when you need to answer questions about employee timekeeping policies.

        <user>: What is the policy on unused vacation leave?
        <assistant>: I need to check the timekeeping policies to answer this question.
        <assistant>: Action: Timekeeping Policies
        <assistant>: Action Input: Vacation Leave Policy - Unused Leave
        ...
        
Employee Data(query) - 
        Useful for when you need to answe

In [11]:
get_response('What is my ID?')



> Entering new AgentExecutor chain...
I need to check the employee data to get this information.
Action: Employee Data
Action Input: df[df['name'] == 'Siva Sai']['employee_id']
Observation: 0    1005674
Name: employee_id, dtype: int64
Thought:I now know the final answer.
Final Answer: Your employee ID is 1005674.

> Finished chain.


{'input': 'What is my ID?', 'output': 'Your employee ID is 1005674.'}

In [64]:
get_response('Are employees on probation allowed to have vacation leaves?')



> Entering new AgentExecutor chain...
I need to check the timekeeping policies to answer this question.
Action: Timekeeping Policies
Action Input: Probationary Period - Vacation Leave Policy

Observation: Based on the provided HR Policy Manual, employees on probation are not eligible for Vacation Leave (A. Vacation Leave, 5. During Probation), but they are eligible for Personal Leave, Sick Leave at a reduced rate, Religious Observance Leave, and Medical Appointment Leave. The eligibility and accrual details for these leaves can be found under sections R, S, and T of the leave policy. The leave application process, including submission, approval, notice period, and documentation requirements, are outlined in section VII of the policy. Violation of this policy may result in disciplinary action, up to and including termination of employment (VIII. Policy Violation). If there are any questions about this policy, the Human Resources Department should be contacted (X. Questions).
Thought:I

{'input': 'Are employees on probation allowed to have vacation leaves?',
 'output': 'Based on the timekeeping policies, employees on probation are not allowed to have vacation leaves. However, they are eligible for Personal Leave, Sick Leave at a reduced rate, Religious Observance Leave, and Medical Appointment Leave.'}

In [65]:
get_response("How many vacation leaves do I have left and what is the policy on unused VLs?")



> Entering new AgentExecutor chain...
I need to check the employee data to get the number of vacation leaves left and the timekeeping policies to understand the policy on unused vacation leaves.

Action: Employee Data
Action Input: df[df['name'] == 'Siva Sai']['vacation_leave']

Observation: 0    30
Name: vacation_leave, dtype: int64
Thought:I now know the number of vacation leaves Siva Sai has left. Next, I need to check the timekeeping policies to understand the policy on unused vacation leaves.

Action: Timekeeping Policies
Action Input: Vacation Leave Policy - Unused Leave

Observation: Based on the provided HR Policy Manual, the Vacation Leave policy states that:

- All regular full-time employees are eligible for Vacation Leave and will earn 1.25 days of leave per month of service, accruing to 15 days per year.
- Unused Vacation Leave can be carried over to the next year, but the total accumulated leave should not exceed 30 days. Any excess leave will be forfeited.
- Unused Vac

{'input': 'How many vacation leaves do I have left and what is the policy on unused VLs?',
 'output': 'You have 30 vacation leaves left. According to the timekeeping policies, unused vacation leaves can be carried over to the next year, but the total accumulated leave should not exceed 30 days. Any excess leave will be forfeited. You can also encash your unused leaves at the end of the year.'}

In [139]:
get_response("How much will Diwakar be paid if he want to encash unused VLs for the month?")



> Entering new AgentExecutor chain...
To answer this question, I need to know Diwakar's basic pay and the number of unused vacation leaves (VLs) he has. I can use the 'Employee Data' tool to get this information.

Action: Employee Data
Action Input: df[df['name'] == 'Diwakar']['basic_pay_in_rs'], df[df['name'] == 'Diwakar']['vacation_leave']

Observation: (1    90000
Name: basic_pay_in_rs, dtype: int64, 1    20
Name: vacation_leave, dtype: int64)
Thought:I now have Diwakar's basic pay (Rs. 90,000) and the number of unused vacation leaves (20). To calculate Diwakar's pay for encashing unused VLs, I need to know the rate at which VLs are encashed. I don't have this information yet, so I will assume it is a fixed rate for the sake of this example. I can use the 'Calculator' tool to perform the calculation.

Action: Calculator
Action Input: (20 * 90000 / 30)


> Entering new LLMMathChain chain...
(20 * 90000 / 30)
```text
20 * 90000 / 30
```
...numexpr.evaluate("20 * 90000 / 30")...

Ans

{'input': 'How much will Diwakar be paid if he want to encash unused VLs for the month?',
 'output': 'If Diwakar wants to encash his unused vacation leaves for the month, he will be paid approximately Rs. 60,000, assuming a fixed rate for encashing vacation leaves.'}

In [62]:
get_response('Which employee has highest pay?')



> Entering new AgentExecutor chain...
I need to check the employee data to find out who has the highest pay.
Action: Employee Data
Action Input: df[['name', 'basic_pay_in_rs']].sort_values(by='basic_pay_in_rs', ascending=False)
Observation:        name  basic_pay_in_rs
3    Thyagu           150000
1   Diwakar            90000
0  Siva Sai            57000
2     Veera            30000
Thought:I now know the final answer.
Final Answer: Thyagu is the employee with the highest pay at Rs. 150,000.

> Finished chain.


{'input': 'Which employee has highest pay?',
 'output': 'Thyagu is the employee with the highest pay at Rs. 150,000.'}

In [12]:
get_response('What is Thyagu basci pay?')



> Entering new AgentExecutor chain...
I need to check the employee data to get this information.
Action: Employee Data
Action Input: df[df['name'] == 'Thyagu']['basic_pay_in_rs']
Observation: 3    150000
Name: basic_pay_in_rs, dtype: int64
Thought:I now know the final answer.
Final Answer: Thyagu's basic pay is 150,000.

> Finished chain.


{'input': 'What is Thyagu basci pay?',
 'output': "Thyagu's basic pay is 150,000."}